In [1]:
import numpy as np 
import math 
import scipy.special as sp

In [2]:
mulmc = 18.477
slmc  = 0.0263
mu4258 = 29.397  # 参见 https://arxiv.org/pdf/1908.05625.pdf
s4258  = 0.032

In [3]:
def ymwcep(datamwcep):
    return np.array([
        row[6] + 0.0075 - 10 + (5 / np.log(10)) * np.log(row[11])
        for row in datamwcep
    ])

def ylmccep(datalmccep):
    return np.array([row[8] + 0.03 for row in datalmccep])

def yhosaccep(datahosaccep):
    return datahosaccep[:1486, 6]

def yhossn(datahossn):
    return np.array([row[4] + row[6] for row in datahossn])

In [4]:
#mwceph=np.genfromtxt('datamw_cep2.txt')
datamwcep = np.genfromtxt("datamw_cep2.txt")
datalmccep = np.genfromtxt( "datalmc_cep.txt")
datahosaccep = np.genfromtxt( "datahosac_cep.txt")
datahossn = np.genfromtxt("datahos_sn.txt")

In [5]:
y_mwcep = ymwcep(datamwcep)
y_hosaccep = yhosaccep(datahosaccep)
y_lmccep = ylmccep(datalmccep)
y_hossn = yhossn(datahossn)

In [6]:
y = np.concatenate([y_mwcep, y_hosaccep, y_lmccep, np.array([mu4258]), np.array([mulmc]), y_hossn])

In [7]:
def dchi(nsig, M):

    y = sp.gammaincinv(M**2, 1 - sp.erf(nsig**2))
    return 2 * y

In [8]:


# --- 参数组合函数 ---
def xpar(rwlmc, rwmw, rw1, rw2, rw3, rw4, rw5, rw6, rw7, rw8, rw9, rw10, rw11, rw12,
         rw13, rw14, rw15, rw16, rw17, rw18, rw19, rw4258, rw21, mu1, mu2, mu3,
         mu4, mu5, mu6, mu7, mu8, mu9, mu10, mu11, mu12, mu13, mu14, mu15, mu16,
         mu17, mu18, mu19, mup4258, mu21, muplmc, mhw, bws, bwl, zw, zp, mb):
    return [rwlmc, rwmw, rw1, rw2, rw3, rw4, rw5, rw6, rw7, rw8, rw9, rw10, rw11, rw12,
            rw13, rw14, rw15, rw16, rw17, rw18, rw19, rw4258, rw21, mu1, mu2, mu3,
            mu4, mu5, mu6, mu7, mu8, mu9, mu10, mu11, mu12, mu13, mu14, mu15, mu16,
            mu17, mu18, mu19, mup4258, mu21, muplmc, mhw, bws, bwl, zw, zp, mb]

# --- 设计矩阵 A（MW 部分） ---
n_mw = len(datamwcep)
# MWr: Table[ datamwcep[[i,3]] - datamwcep[[i,5]], {i,1,n_mw} ]
MWr = np.array([[row[2] - row[4]] for row in datamwcep])
MW1 = np.zeros((n_mw, 43))
MW2 = np.ones((n_mw, 1))
MW3 = np.zeros((n_mw, 1))
# psmwcep: Table[ If[datamwcep[[i,2]] < 1, datamwcep[[i,2]] - 1, 0], {i,1,n_mw} ]
psmwcep = np.array([[row[1] - 1 if row[1] < 1 else 0] for row in datamwcep])
# plmwcep: Table[ If[datamwcep[[i,2]] > 1, datamwcep[[i,2]] - 1, 0], {i,1,n_mw} ]
plmwcep = np.array([[row[1] - 1 if row[1] > 1 else 0] for row in datamwcep])
# mhmwcep: Table[ datamwcep[[i,11]], {i,1,n_mw} ]
mhmwcep = np.array([[row[10]] for row in datamwcep])
# pimwcep: Table[ -5/np.log(10)*Log[datamwcep[[i,12]]], {i,1,n_mw} ]
pimwcep = np.array([[-5 / np.log10(row[11])] for row in datamwcep])
# 将各块沿列拼接（相当于 Mathematica 中 Join[...,2]）
MWall = np.hstack([MW3, MWr, MW1, MW2, psmwcep, plmwcep, mhmwcep, pimwcep, MW3])
print("MWall shape:", MWall.shape)

# === 以下对 datahosaccep 构造 G 系列矩阵 ===
# 注：datahosaccep 中各块取法均需注意：Mathematica 中 i 的取值范围转换为 Python 索引时均减 1。

# --- G1 block (i = 1 to 251) ---
n_G1 = 251
G1r = np.array([[datahosaccep[i, 5]] for i in range(0, n_G1)])
G1r01 = np.zeros((n_G1, 2))
G1r02 = np.zeros((n_G1, 20))
G11 = np.ones((n_G1, 1))
G12 = np.zeros((n_G1, 21))
G13 = np.zeros((n_G1, 2))
psg1cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(0, n_G1)])
plg1cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(0, n_G1)])
mhg1cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(0, n_G1)])
G1all = np.hstack([G1r01, G1r, G1r02, G11, G12, G11, psg1cep, plg1cep, mhg1cep, G13])
print("G1all shape:", G1all.shape)

# --- G2 block (i = 252 to 265) ---
n_G2 = 265 - 252 + 1  # 14 rows
G2r = np.array([[datahosaccep[i, 5]] for i in range(251, 265)])
G2r01 = np.zeros((n_G2, 3))
G2r02 = np.zeros((n_G2, 19))
G21 = np.zeros((n_G2, 1))
G22 = np.ones((n_G2, 1))
G23 = np.zeros((n_G2, 20))
G24 = np.zeros((n_G2, 2))
psg2cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(251, 265)])
plg2cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(251, 265)])
mhg2cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(251, 265)])
G2all = np.hstack([G2r01, G2r, G2r02, G21, G22, G23, G22, psg2cep, plg2cep, mhg2cep, G24])
print("G2all shape:", G2all.shape)

# --- G3 block (i = 266 to 309) ---
n_G3 = 309 - 266 + 1  # 44 rows
G3r = np.array([[datahosaccep[i, 5]] for i in range(265, 309)])
G3r01 = np.zeros((n_G3, 4))
G3r02 = np.zeros((n_G3, 18))
G31 = np.zeros((n_G3, 2))
G32 = np.ones((n_G3, 1))
G33 = np.zeros((n_G3, 19))
G34 = np.zeros((n_G3, 2))
psg3cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(265, 309)])
plg3cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(265, 309)])
mhg3cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(265, 309)])
G3all = np.hstack([G3r01, G3r, G3r02, G31, G32, G33, G32, psg3cep, plg3cep, mhg3cep, G34])
print("G3all shape:", G3all.shape)

# --- G4 block (i = 310 to 341) ---
n_G4 = 341 - 310 + 1  # 32 rows
G4r = np.array([[datahosaccep[i, 5]] for i in range(309, 341)])
G4r01 = np.zeros((n_G4, 5))
G4r02 = np.zeros((n_G4, 17))
G41 = np.zeros((n_G4, 3))
G42 = np.ones((n_G4, 1))
G43 = np.zeros((n_G4, 18))
G44 = np.zeros((n_G4, 2))
psg4cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(309, 341)])
plg4cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(309, 341)])
mhg4cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(309, 341)])
G4all = np.hstack([G4r01, G4r, G4r02, G41, G42, G43, G42, psg4cep, plg4cep, mhg4cep, G44])
print("G4all shape:", G4all.shape)

# --- G5 block (i = 342 to 395) ---
n_G5 = 395 - 342 + 1  # 54 rows
G5r = np.array([[datahosaccep[i, 5]] for i in range(341, 395)])
G5r01 = np.zeros((n_G5, 6))
G5r02 = np.zeros((n_G5, 16))
G51 = np.zeros((n_G5, 4))
G52 = np.ones((n_G5, 1))
G53 = np.zeros((n_G5, 17))
G54 = np.zeros((n_G5, 2))
psg5cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(341, 395)])
plg5cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(341, 395)])
mhg5cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(341, 395)])
G5all = np.hstack([G5r01, G5r, G5r02, G51, G52, G53, G52, psg5cep, plg5cep, mhg5cep, G54])
print("G5all shape:", G5all.shape)

# --- G6 block (i = 396 to 536) ---
n_G6 = 536 - 396 + 1  # 141 rows
G6r = np.array([[datahosaccep[i, 5]] for i in range(395, 536)])
G6r01 = np.zeros((n_G6, 7))
G6r02 = np.zeros((n_G6, 15))
G61 = np.zeros((n_G6, 5))
G62 = np.ones((n_G6, 1))
G63 = np.zeros((n_G6, 16))
G64 = np.zeros((n_G6, 2))
psg6cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(395, 536)])
plg6cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(395, 536)])
mhg6cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(395, 536)])
G6all = np.hstack([G6r01, G6r, G6r02, G61, G62, G63, G62, psg6cep, plg6cep, mhg6cep, G64])
print("G6all shape:", G6all.shape)

# --- G7 block (i = 537 to 554) ---
n_G7 = 554 - 537 + 1  # 18 rows
G7r = np.array([[datahosaccep[i, 5]] for i in range(536, 554)])
G7r01 = np.zeros((n_G7, 8))
G7r02 = np.zeros((n_G7, 14))
G71 = np.zeros((n_G7, 6))
G72 = np.ones((n_G7, 1))
G73 = np.zeros((n_G7, 15))
G74 = np.zeros((n_G7, 2))
psg7cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(536, 554)])
plg7cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(536, 554)])
mhg7cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(536, 554)])
G7all = np.hstack([G7r01, G7r, G7r02, G71, G72, G73, G72, psg7cep, plg7cep, mhg7cep, G74])
print("G7all shape:", G7all.shape)

# --- G8 block (i = 555 to 617) ---
n_G8 = 617 - 555 + 1  # 63 rows
G8r = np.array([[datahosaccep[i, 5]] for i in range(554, 617)])
G8r01 = np.zeros((n_G8, 9))
G8r02 = np.zeros((n_G8, 13))
G81 = np.zeros((n_G8, 7))
G82 = np.ones((n_G8, 1))
G83 = np.zeros((n_G8, 14))
G84 = np.zeros((n_G8, 2))
psg8cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(554, 617)])
plg8cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(554, 617)])
mhg8cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(554, 617)])
G8all = np.hstack([G8r01, G8r, G8r02, G81, G82, G83, G82, psg8cep, plg8cep, mhg8cep, G84])
print("G8all shape:", G8all.shape)

# --- G9 block (i = 618 to 697) ---
n_G9 = 697 - 618 + 1  # 80 rows
G9r = np.array([[datahosaccep[i, 5]] for i in range(617, 697)])
G9r01 = np.zeros((n_G9, 10))
G9r02 = np.zeros((n_G9, 12))
G91 = np.zeros((n_G9, 8))
G92 = np.ones((n_G9, 1))
G93 = np.zeros((n_G9, 13))
G94 = np.zeros((n_G9, 2))
psg9cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(617, 697)])
plg9cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(617, 697)])
mhg9cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(617, 697)])
G9all = np.hstack([G9r01, G9r, G9r02, G91, G92, G93, G92, psg9cep, plg9cep, mhg9cep, G94])
print("G9all shape:", G9all.shape)

# --- G10 block (i = 698 to 739) ---
n_G10 = 739 - 698 + 1  # 42 rows
G10r = np.array([[datahosaccep[i, 5]] for i in range(697, 739)])
G10r01 = np.zeros((n_G10, 11))
G10r02 = np.zeros((n_G10, 11))
G101 = np.zeros((n_G10, 9))
G102 = np.ones((n_G10, 1))
G103 = np.zeros((n_G10, 12))
G104 = np.zeros((n_G10, 2))
psg10cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(697, 739)])
plg10cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(697, 739)])
mhg10cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(697, 739)])
G10all = np.hstack([G10r01, G10r, G10r02, G101, G102, G103, G102, psg10cep, plg10cep, mhg10cep, G104])
print("G10all shape:", G10all.shape)

# --- G11 block (i = 740 to 755) ---
n_G11 = 755 - 740 + 1  # 16 rows
G11r = np.array([[datahosaccep[i, 5]] for i in range(739, 755)])
G11r01 = np.zeros((n_G11, 12))
G11r02 = np.zeros((n_G11, 10))
G111 = np.zeros((n_G11, 10))
G112 = np.ones((n_G11, 1))
G113 = np.zeros((n_G11, 11))
G114 = np.zeros((n_G11, 2))
psg11cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(739, 755)])
plg11cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(739, 755)])
mhg11cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(739, 755)])
G11all = np.hstack([G11r01, G11r, G11r02, G111, G112, G113, G112, psg11cep, plg11cep, mhg11cep, G114])
print("G11all shape:", G11all.shape)

# --- G12 block (i = 756 to 768) ---
n_G12 = 768 - 756 + 1  # 13 rows
G12r = np.array([[datahosaccep[i, 5]] for i in range(755, 768)])
G12r01 = np.zeros((n_G12, 13))
G12r02 = np.zeros((n_G12, 9))
G121 = np.zeros((n_G12, 11))
G122 = np.ones((n_G12, 1))
G123 = np.zeros((n_G12, 10))
G124 = np.zeros((n_G12, 2))
psg12cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(755, 768)])
plg12cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(755, 768)])
mhg12cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(755, 768)])
G12all = np.hstack([G12r01, G12r, G12r02, G121, G122, G123, G122, psg12cep, plg12cep, mhg12cep, G124])
print("G12all shape:", G12all.shape)

# --- G13 block (i = 769 to 771) ---
n_G13 = 771 - 769 + 1  # 3 rows
G13r = np.array([[datahosaccep[i, 5]] for i in range(768, 771)])
G13r01 = np.zeros((n_G13, 14))
G13r02 = np.zeros((n_G13, 8))
G131 = np.zeros((n_G13, 12))
G132 = np.ones((n_G13, 1))
G133 = np.zeros((n_G13, 9))
G134 = np.zeros((n_G13, 2))
psg13cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(768, 771)])
plg13cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(768, 771)])
mhg13cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(768, 771)])
G13all = np.hstack([G13r01, G13r, G13r02, G131, G132, G133, G132, psg13cep, plg13cep, mhg13cep, G134])
print("G13all shape:", G13all.shape)

# --- G14 block (i = 772 to 804) ---
n_G14 = 804 - 772 + 1  # 33 rows
G14r = np.array([[datahosaccep[i, 5]] for i in range(771, 804)])
G14r01 = np.zeros((n_G14, 15))
G14r02 = np.zeros((n_G14, 7))
G141 = np.zeros((n_G14, 13))
G142 = np.ones((n_G14, 1))
G143 = np.zeros((n_G14, 8))
G144 = np.zeros((n_G14, 2))
psg14cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(771, 804)])
plg14cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(771, 804)])
mhg14cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(771, 804)])
G14all = np.hstack([G14r01, G14r, G14r02, G141, G142, G143, G142, psg14cep, plg14cep, mhg14cep, G144])
print("G14all shape:", G14all.shape)

# --- G15 block (i = 805 to 829) ---
n_G15 = 829 - 805 + 1  # 25 rows
G15r = np.array([[datahosaccep[i, 5]] for i in range(804, 829)])
G15r01 = np.zeros((n_G15, 16))
G15r02 = np.zeros((n_G15, 6))
G151 = np.zeros((n_G15, 14))
G152 = np.ones((n_G15, 1))
G153 = np.zeros((n_G15, 7))
G154 = np.zeros((n_G15, 2))
psg15cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(804, 829)])
plg15cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(804, 829)])
mhg15cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(804, 829)])
G15all = np.hstack([G15r01, G15r, G15r02, G151, G152, G153, G152, psg15cep, plg15cep, mhg15cep, G154])
print("G15all shape:", G15all.shape)

# --- G16 block (i = 830 to 912) ---
n_G16 = 912 - 830 + 1  # 83 rows
G16r = np.array([[datahosaccep[i, 5]] for i in range(829, 912)])
G16r01 = np.zeros((n_G16, 17))
G16r02 = np.zeros((n_G16, 5))
G161 = np.zeros((n_G16, 15))
G162 = np.ones((n_G16, 1))
G163 = np.zeros((n_G16, 6))
G164 = np.zeros((n_G16, 2))
psg16cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(829, 912)])
plg16cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(829, 912)])
mhg16cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(829, 912)])
G16all = np.hstack([G16r01, G16r, G16r02, G161, G162, G163, G162, psg16cep, plg16cep, mhg16cep, G164])
print("G16all shape:", G16all.shape)

# --- G17 block (i = 913 to 925) ---
n_G17 = 925 - 913 + 1  # 13 rows
G17r = np.array([[datahosaccep[i, 5]] for i in range(912, 925)])
G17r01 = np.zeros((n_G17, 18))
G17r02 = np.zeros((n_G17, 4))
G171 = np.zeros((n_G17, 16))
G172 = np.ones((n_G17, 1))
G173 = np.zeros((n_G17, 5))
G174 = np.zeros((n_G17, 2))
psg17cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(912, 925)])
plg17cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(912, 925)])
mhg17cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(912, 925)])
G17all = np.hstack([G17r01, G17r, G17r02, G171, G172, G173, G172, psg17cep, plg17cep, mhg17cep, G174])
print("G17all shape:", G17all.shape)

# --- G18 block (i = 926 to 947) ---
n_G18 = 947 - 926 + 1  # 22 rows
G18r = np.array([[datahosaccep[i, 5]] for i in range(925, 947)])
G18r01 = np.zeros((n_G18, 19))
G18r02 = np.zeros((n_G18, 3))
G181 = np.zeros((n_G18, 17))
G182 = np.ones((n_G18, 1))
G183 = np.zeros((n_G18, 4))
G184 = np.zeros((n_G18, 2))
psg18cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(925, 947)])
plg18cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(925, 947)])
mhg18cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(925, 947)])
G18all = np.hstack([G18r01, G18r, G18r02, G181, G182, G183, G182, psg18cep, plg18cep, mhg18cep, G184])
print("G18all shape:", G18all.shape)

# --- G19 block (i = 948 to 975) ---
n_G19 = 975 - 948 + 1  # 28 rows
G19r = np.array([[datahosaccep[i, 5]] for i in range(947, 975)])
G19r01 = np.zeros((n_G19, 20))
G19r02 = np.zeros((n_G19, 2))
G191 = np.zeros((n_G19, 18))
G192 = np.ones((n_G19, 1))
G193 = np.zeros((n_G19, 3))
G194 = np.zeros((n_G19, 2))
psg19cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(947, 975)])
plg19cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(947, 975)])
mhg19cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(947, 975)])
G19all = np.hstack([G19r01, G19r, G19r02, G191, G192, G193, G192, psg19cep, plg19cep, mhg19cep, G194])
print("G19all shape:", G19all.shape)

# --- G20 block (i = 976 to 1114) ---
n_G20 = 1114 - 976 + 1  # 139 行

# 提取数据列
G20r = np.array([[datahosaccep[i, 5]] for i in range(976, 1115)])  # 第6列 (索引5)
G20r01 = np.zeros((n_G20, 21))  # 139×21 全零矩阵
G20r02 = np.zeros((n_G20, 1))   # 139×1  全零矩阵
G42581 = np.zeros((n_G20, 19))  # 139×19 全零矩阵
G42582 = np.ones((n_G20, 1))    # 139×1  全1矩阵
G42583 = np.zeros((n_G20, 2))   # 139×2  全零矩阵
G42584 = np.zeros((n_G20, 2))   # 139×2  全零矩阵

# 计算 psg4258cep（如果 datahosaccep[i,4] < 10，则取 log10(datahosaccep[i,4]) - 1，否则为 0）
psg4258cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(976, 1115)])

# 计算 plg4258cep（如果 datahosaccep[i,4] > 10，则取 log10(datahosaccep[i,4]) - 1，否则为 0）
plg4258cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(976, 1115)])

# 计算 mhg4258cep（datahosaccep[i,8] - 8.824）
mhg4258cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(976, 1115)])

# 拼接矩阵 (axis=1 代表按列拼接，相当于 Mathematica 的 Join[..., 2])
G4258all = np.hstack([G20r01, G20r, G20r02, G42581, G42582, G42583, G42582, psg4258cep, plg4258cep, mhg4258cep, G42584])

# 输出矩阵形状检查
print("G4258all shape:", G4258all.shape)


# --- G21 block (i = 1115 to 1486) ---
n_G21 = 1486 - 1115 + 1  # 372 rows
G21r = np.array([[datahosaccep[i, 5]] for i in range(1114, 1486)])
G21r01 = np.zeros((n_G21, 22))
G211 = np.zeros((n_G21, 20))
G212 = np.ones((n_G21, 1))
G213 = np.zeros((n_G21, 1))
G214 = np.zeros((n_G21, 2))
psg21cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] < 10 else 0] for i in range(1114, 1486)])
plg21cep = np.array([[(np.log10(datahosaccep[i, 4]) - 1) if datahosaccep[i, 4] > 10 else 0] for i in range(1114, 1486)])
mhg21cep = np.array([[datahosaccep[i, 8] - 8.824] for i in range(1114, 1486)])
G21all = np.hstack([G21r01, G21r, G211, G212, G213, G212, psg21cep, plg21cep, mhg21cep, G214])
print("G21all shape:", G21all.shape)

# === Glmc cepheids部分 ===
# Glmcr: Table[datalmccep[[i,5]] - datalmccep[[i,7]] , {i,1,70}]
n_glmc = 70
Glmcr = np.array([[datalmccep[i, 4] - datalmccep[i, 6]] for i in range(n_glmc)])
Glmc1 = np.zeros((n_glmc, 43))
Glmc2 = np.ones((n_glmc, 1))
Glmc3 = np.zeros((n_glmc, 1))
# psglmccep: Table[ If[datalmccep[[i,4]] < 1, datalmccep[[i,4]] - 1, 0], {i,1,70} ]
psglmccep = np.array([[datalmccep[i, 3] - 1 if datalmccep[i, 3] < 1 else 0] for i in range(n_glmc)])
plglmccep = np.array([[datalmccep[i, 3] - 1 if datalmccep[i, 3] > 1 else 0] for i in range(n_glmc)])
mhglmccep = np.full((n_glmc, 1), -0.3)
Glmc4 = np.zeros((n_glmc, 2))
Glmcall = np.hstack([Glmcr, Glmc1, Glmc2, Glmc2, psglmccep, plglmccep, mhglmccep, Glmc4])
print("Glmcall shape:", Glmcall.shape)

# === Gmur 及 SN 系列 ===
# Gmur: 1×23 零向量
Gmur = np.zeros((1, 23))
# Gmu4258: 将 Gmur 与一个 1×N 的矩阵拼接，这里根据原代码构造示例（具体位置的 1 可调整）
Gmu4258 = np.hstack([Gmur, np.array([[0]*19 + [1] + [0]*8])])
# Gmulmc: 类似构造
Gmulmc = np.hstack([Gmur, np.array([[0]*21 + [1] + [0]*6])])
# SN 系列：每个 SNi 均由 Gmur 拼接一个 1×23 的“1-hot”向量构成
SN1  = np.hstack([Gmur, np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN2  = np.hstack([Gmur, np.array([[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN3  = np.hstack([Gmur, np.array([[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN4  = np.hstack([Gmur, np.array([[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN5  = np.hstack([Gmur, np.array([[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN6  = np.hstack([Gmur, np.array([[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN7  = np.hstack([Gmur, np.array([[0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN8  = np.hstack([Gmur, np.array([[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN9  = np.hstack([Gmur, np.array([[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN10 = np.hstack([Gmur, np.array([[0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN11 = np.hstack([Gmur, np.array([[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN12 = np.hstack([Gmur, np.array([[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN13 = np.hstack([Gmur, np.array([[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN14 = np.hstack([Gmur, np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN15 = np.hstack([Gmur, np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN16 = np.hstack([Gmur, np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1]])])
SN17 = np.hstack([Gmur, np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1]])])
SN18 = np.hstack([Gmur, np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1]])])
SN19 = np.hstack([Gmur, np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1]])])
print("Gmu4258 shape:", Gmu4258.shape)
print("Gmulmc shape:", Gmulmc.shape)
print("SN1 shape:", SN1.shape)
# 可依此打印其他 SNi 的形状

if __name__ == "__main__":
    # 示例调用
    print("dchi(1,2) =", dchi(1, 2))


MWall shape: (74, 51)
G1all shape: (251, 51)
G2all shape: (14, 51)
G3all shape: (44, 51)
G4all shape: (32, 51)
G5all shape: (54, 51)
G6all shape: (141, 51)
G7all shape: (18, 51)
G8all shape: (63, 51)
G9all shape: (80, 51)
G10all shape: (42, 51)
G11all shape: (16, 51)
G12all shape: (13, 51)
G13all shape: (3, 51)
G14all shape: (33, 51)
G15all shape: (25, 51)
G16all shape: (83, 51)
G17all shape: (13, 51)
G18all shape: (22, 51)
G19all shape: (28, 51)
G4258all shape: (139, 51)
G21all shape: (372, 51)
Glmcall shape: (70, 51)
Gmu4258 shape: (1, 51)
Gmulmc shape: (1, 51)
SN1 shape: (1, 51)
dchi(1,2) = 4.156860350537519


In [9]:
A = np.vstack([
    MWall, G1all, G2all, G3all, G4all, G5all, G6all, G7all, G8all, G9all,
    G10all, G11all, G12all, G13all, G14all, G15all, G16all, G17all, G18all, G19all,
    G4258all, G21all, Glmcall, Gmu4258, Gmulmc,
    SN1, SN2, SN3, SN4, SN5, SN6, SN7, SN8, SN9, SN10,
    SN11, SN12, SN13, SN14, SN15, SN16, SN17, SN18, SN19
])

print("A shape:", A.shape)

A shape: (1651, 51)


In [10]:
import numpy as np

# 参数定义
slmcb = 0.08
s4258 = 0.032  # 之前提供的 s4258 值
slmc = 0.0263  # 之前提供的 slmc 值

# 获取数据的长度
n_mw = len(datamwcep)        # MW 数据点数
n_hosac = 1486               # HOSAC 数据点数
n_lmc = 70                   # LMC 数据点数
n_4258_lmc = 2               # 4258 + LMC 数据点数
n_sn = 19                    # SN 数据点数
total_cols = 1650            # 最终 C 矩阵的总列数

# --- CMW 矩阵 ---
CMW1 = np.diag([
    (row[7] + 0.006)**2 + ((5 / np.log(10)) * (row[12] / row[11]))**2 +
    0.39**2 * (row[3]**2 + row[5]**2)
    for row in datamwcep
])

CMW2 = np.zeros((n_mw, total_cols - n_mw + 1 ))


CMW = np.hstack([CMW1, CMW2])  # Join[..., 2] 转换为 np.hstack

print("CMW shape:", CMW.shape)

# --- CHOSAC 矩阵 ---
CHOSAC1 = np.zeros((n_hosac, n_mw))
CHOSAC2 = np.diag([row[7]**2 for row in datahosaccep[:n_hosac]])
CHOSAC3 = np.zeros((n_hosac, total_cols - 73 - n_hosac))

CHOSAC = np.hstack([CHOSAC1, CHOSAC2, CHOSAC3])

print("CHOSAC shape:", CHOSAC.shape)

# --- CLMC 矩阵 ---
CLMC1 = np.zeros((n_lmc, n_mw + n_hosac))

CLMC2 = np.diag([
    ((row[9] + 0.024)**2 + slmcb**2 + 0.39**2 * (row[5]**2 + row[7]**2))
    for row in datalmccep[:n_lmc]
])

CLMC3 = np.zeros((n_lmc, total_cols - 143 - n_hosac))

CLMC = np.hstack([CLMC1, CLMC2, CLMC3])

print("CLMC shape:", CLMC.shape)

# --- C4258LMC 矩阵 ---
C4258LMC1 = np.zeros((n_4258_lmc, n_mw + 1114 + 372 + n_lmc))

C4258LMC2 = np.diag([s4258**2, slmc**2])

C4258LMC3 = np.zeros((n_4258_lmc, 19))

C4258LMC = np.hstack([C4258LMC1, C4258LMC2, C4258LMC3])

print("C4258LMC shape:", C4258LMC.shape)

# --- CSN 矩阵 ---
CSN1 = np.zeros((n_sn, n_mw + 1114 + 372 + n_lmc + 2))

CSN2 = np.diag([row[3]**2 for row in datahossn[:n_sn]])

CSN = np.hstack([CSN1, CSN2])

print("CSN shape:", CSN.shape)

# --- Call 矩阵 ---
Call = np.vstack([CMW, CHOSAC, CLMC, C4258LMC, CSN])

print("Call shape:", Call.shape)


CMW shape: (74, 1651)
CHOSAC shape: (1486, 1651)
CLMC shape: (70, 1651)
C4258LMC shape: (2, 1651)
CSN shape: (19, 1651)
Call shape: (1651, 1651)


In [14]:
InvCall = np.linalg.inv(Call)  # Inverse[Call]
traA = A.T                     # Transpose[A]
sigma = np.linalg.inv(traA @ InvCall @ A)
xbf = sigma @ (traA @ InvCall @ y)
param_errors = np.sqrt(np.diag(sigma))

In [15]:
ab=0.71273
sab=0.00176
h0=10**(0.2* xbf[-1]+ab+5)
h0

68.90322450187507

In [17]:
sh0=h0 * np.log(10) * np.sqrt(0.2**2 * param_errors[-1]**2 + sab**2)
sh0

2.470164852337431